In [1]:
import pandas as pd
import numpy as np
import preprocessor as p
import demoji

#NLTK

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer, LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize

# Define a variable for stopwords search from NLTK

stop_words = set(nltk.corpus.stopwords.words('english'))

In [2]:
# Read dataset

file_path = 'C:\\Thesis\\Twittter\\220315 - Top 100 Airports Twitter 100 Airports Data 22 Dec 2021 - 14 Mar 2022.xlsx'

twitter = pd.read_excel(file_path)

## Observation

In [3]:
twitter.head(10)

,Date,Screen Name,Full Name,Tweet Text,Tweet ID,App,Followers,Follows,Retweets,Favorites,Verfied,User Since,Location,Bio,Profile Image,Google Maps
0,2021-12-22 20:24:43,@BruinUpSomeChou,Vincent Chou,@Airport_FRA Lost my wallet and even though it...,1473668328401084426,Twitter for Android,33,109,0,0,No,2011-12-23 08:18:03,"Dallas, TX",I'm bad at Twitter,View,NaN
1,2021-12-22 20:37:49,@OmeirHumble,Khan Sahab,@fly2ohare @thepointsguy 😂😂😂,1473671624188735491,Twitter for iPhone,19,570,0,0,No,2020-08-20 01:10:25,"Dubai, United Arab Emirates",An ordinary person lucky enough to have lived ...,View,NaN
2,2021-12-22 20:37:55,@fly2midway,Midway Intl. Airport,@fly2ohare @thepointsguy 😉 https://t.co/sBlF7S...,1473671648834318337,Twitter for iPhone,20867,219,1,6,No,2011-10-03 22:53:50,"Chicago, IL USA",The official Midway International Airport twit...,View,NaN
3,2021-12-22 20:38:46,@haysdewy,haysdewy,RT @fly2midway: @fly2ohare @thepointsguy 😉 htt...,1473671865658920960,Twitter Web App,30,287,1,0,No,2021-01-28 22:25:02,"Soo Foo, So Dak 57110",Christian softball natural mama & wifey. Retir...,View,NaN
4,2021-12-22 20:40:16,@lemahevad,Dave in Cedar Rapids,"@fly2ohare @thepointsguy Ooh, how do I say thi...",1473672241040830465,Twitter Web App,167,722,0,0,No,2008-12-02 21:57:00,"CID, more often than not",Let me tell you why your airplane is broken. S...,View,NaN
5,2021-12-22 20:48:41,@Jeffa9311,Jeffa,@fly2ohare @thepointsguy https://t.co/ltODWTBklb,1473674360636551176,Twitter for Android,67,489,0,0,No,2014-05-29 19:41:18,The Moon,Amateur golfer and gambler. Professional insti...,View,NaN
6,2021-12-22 21:11:41,@KKIASA,RUH |مطار الملك خالد,"Your passport is your identity when traveling,...",1473680148226621442,Twitter for iPhone,263541,12,2,3,No,2015-07-16 23:23:29,Riyadh,الحساب الرسمي لمطار الملك خالد الدولي بمدينة ا...,View,NaN
7,2021-12-22 21:12:00,@bkuuunks,BK 👑,@fly2ohare @thepointsguy https://t.co/ahvsY8QP5q,1473680228706889748,Twitter for iPhone,153,194,0,0,No,2011-05-05 14:40:12,USA,NaN,View,NaN
8,2021-12-22 21:12:45,@oPekAK133kyw5i7,ابو فيصل,@KKIASA *لـقـاح كـارونـا بـدون حـضـور* *الـجـر...,1473680414283870217,Twitter for Android,79,306,0,0,No,2021-08-19 10:43:08,المملكة العربية السعودية,مكتب للخدمات العامه,View,NaN
9,2021-12-22 21:17:28,@cecinchilla_,Cecilia⁷,@MiAirports Buonasera. Come devo comportarmi s...,1473681602869960714,Twitter Web App,1201,704,0,0,No,2020-06-05 23:27:17,she/her,ceci you are the reason why men are gay. — Fox...,View,NaN


In [4]:
type(twitter)

pandas.core.frame.DataFrame

In [5]:
len(twitter)

40124

In [6]:
twitter.shape

(40124, 16)

In [7]:
twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40124 entries, 0 to 40123
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           40124 non-null  datetime64[ns]
 1   Screen Name    40124 non-null  object        
 2   Full Name      40118 non-null  object        
 3   Tweet Text     40124 non-null  object        
 4   Tweet ID       40124 non-null  int64         
 5   App            40124 non-null  object        
 6   Followers      40124 non-null  int64         
 7   Follows        40124 non-null  int64         
 8   Retweets       40124 non-null  int64         
 9   Favorites      40124 non-null  int64         
 10  Verfied        40124 non-null  object        
 11  User Since     40124 non-null  datetime64[ns]
 12  Location       24872 non-null  object        
 13  Bio            28159 non-null  object        
 14  Profile Image  40124 non-null  object        
 15  Google Maps    1 no

In [8]:
print(sum(twitter['Tweet Text'].isnull()))

0


## Preprocessing

### Basic cleaning of tweets

In [9]:
# Create a list with tweet texts

text = list(twitter['Tweet Text'])
text[:10]

['@Airport_FRA Lost my wallet and even though it was found (announced over intercom but could not pick up because of… https://t.co/Cw9RUO2Ou0',
 '@fly2ohare @thepointsguy 😂😂😂',
 '@fly2ohare @thepointsguy 😉 https://t.co/sBlF7SBcT8',
 'RT @fly2midway: @fly2ohare @thepointsguy 😉 https://t.co/sBlF7SBcT8',
 '@fly2ohare @thepointsguy Ooh, how do I say this gently...',
 '@fly2ohare @thepointsguy https://t.co/ltODWTBklb',
 'Your passport is your identity when traveling, So keep it safe. #KKIA🛂',
 '@fly2ohare @thepointsguy https://t.co/ahvsY8QP5q',
 '@KKIASA *لـقـاح كـارونـا بـدون حـضـور* *الـجـرعـه الاولـه* *الـجـرعـه الثانيه* *الـجـرعـه الـثالـثه* *( شــغــال… https://t.co/gjT21qZfGR',
 "@MiAirports Buonasera. Come devo comportarmi se voglio trasportare un'arma giocattolo in aereo? é fatta di plastica… https://t.co/OAjUZ0YNGZ"]

In [10]:
# Filter out mentions, URLs, hashtags and s.f. using Preprocessor

def preprocess_tweets(text):
    preprocessed_tweets = [p.clean(tweet) for tweet in text]
    return preprocessed_tweets

processed_tweets = preprocess_tweets(text)

#Remove short messages containing less than 3 characters or not containing characters at all

filtered_tweets = [tweet for tweet in processed_tweets if len(tweet) >= 3 and any(char.isalpha() for char in tweet)]

filtered_tweets[:20]

['Lost my wallet and even though it was found (announced over intercom but could not pick up because of',
 'Ooh, how do I say this gently...',
 'Your passport is your identity when traveling, So keep it safe.',
 "Buonasera. Come devo comportarmi se voglio trasportare un'arma giocattolo in aereo? fatta di plastica",
 'What was the max TSA time yesterday?',
 'KSFO All the way United is the best airline and it is a hub at the Best airport and the corbra tower helps them with ATC!',
 'off to Oahu for a few days.',
 "My inquiry is V14331/21. I don't mean to be pushy, but I am unable to drive since it has my driver's l",
 'The tree decorated by features a colorful mix of ornaments- including DIY bulb ornaments with staff name',
 'One day Ill try to miss a flight in your airport, too!',
 'Evite vestir con objetos metlicos, ahorrar tiempo al pasar por los filtros de seguridad en aero',
 'merry christmas',
 'ohh this is neat!',
 'Sure thing!',
 'Oh Matilda',
 'BCL - Barcelona',
 'or maybe go di

In [11]:
# Check the number of tweets after filtering

len(filtered_tweets)

34224

In [12]:
# Detect tweets written in English with the langdetect module

from langdetect import detect

def is_english(english_tweet):
    english_t = [t for t in english_tweet if detect(t) == 'en']
    return english_t

english_tweets = is_english(filtered_tweets)

# check the number of English-written messages

len(english_tweets)

24271

In [13]:
english_tweets[:20]

['Lost my wallet and even though it was found (announced over intercom but could not pick up because of',
 'Ooh, how do I say this gently...',
 'Your passport is your identity when traveling, So keep it safe.',
 'What was the max TSA time yesterday?',
 'KSFO All the way United is the best airline and it is a hub at the Best airport and the corbra tower helps them with ATC!',
 'off to Oahu for a few days.',
 "My inquiry is V14331/21. I don't mean to be pushy, but I am unable to drive since it has my driver's l",
 'The tree decorated by features a colorful mix of ornaments- including DIY bulb ornaments with staff name',
 'One day Ill try to miss a flight in your airport, too!',
 'merry christmas',
 'ohh this is neat!',
 'Sure thing!',
 'or maybe go direct from Really bored with your Lufthansa shit now',
 'Get & get your booster shot! is hosting a COVID-19 vaccination clinic in partnership with',
 'Probably Spend more money on the dogs than the Custodians',
 'Precious!',
 'Your airport is

In [14]:
processed_twitter = pd.DataFrame(english_tweets, columns=['Original_Tweets'])

processed_twitter.head(10)

,Original_Tweets
0,Lost my wallet and even though it was found (a...
1,"Ooh, how do I say this gently..."
2,"Your passport is your identity when traveling,..."
3,What was the max TSA time yesterday?
4,KSFO All the way United is the best airline an...
5,off to Oahu for a few days.
6,My inquiry is V14331/21. I don't mean to be pu...
7,The tree decorated by features a colorful mix ...
8,One day Ill try to miss a flight in your airpo...
9,merry christmas


### Lemmatization

In [15]:
# Check if NLTK corpora resources are present, download if not

def check_nltk():
    try:
        nltk.data.find('tokenizers/punkt')
        nltk.data.find('taggers/averaged_perceptron_tagger')
        nltk.data.find('corpora/wordnet')
        print("NLTK resources are downloaded.")
    except LookupError:
        print("NLTK resources are not downloaded. Downloading...")
        nltk.download('punkt')
        nltk.download('averaged_perceptron_tagger')
        nltk.download('wordnet')
        print("NLTK resources downloaded successfully.")

check_nltk()

NLTK resources are not downloaded. Downloading...
NLTK resources downloaded successfully.


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anash\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\anash\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anash\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
# Define lemmatization function
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    tokens = nltk.word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(tag))
                         for token, tag in nltk.pos_tag(tokens)]
    return ' '.join(lemmatized_tokens)

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun if no specific POS tag is found

# Apply lemmatization to the 'Original_Tweets' column
processed_twitter['Lemm_Tweets'] = processed_twitter['Original_Tweets'].apply(lemmatize_text)

# Display the updated DataFrame
processed_twitter.head(10)

,Original_Tweets,Lemm_Tweets
0,Lost my wallet and even though it was found (a...,Lost my wallet and even though it be find ( an...
1,"Ooh, how do I say this gently...","Ooh , how do I say this gently ..."
2,"Your passport is your identity when traveling,...",Your passport be your identity when traveling ...
3,What was the max TSA time yesterday?,What be the max TSA time yesterday ?
4,KSFO All the way United is the best airline an...,KSFO All the way United be the best airline an...
5,off to Oahu for a few days.,off to Oahu for a few day .
6,My inquiry is V14331/21. I don't mean to be pu...,My inquiry be V14331/21 . I do n't mean to be ...
7,The tree decorated by features a colorful mix ...,The tree decorate by feature a colorful mix of...
8,One day Ill try to miss a flight in your airpo...,One day Ill try to miss a flight in your airpo...
9,merry christmas,merry christmas


## Topic Modelling with LDA (NLTK)

In [17]:
from gensim import corpora, models
from pprint import pprint
import string

# Convert the column to the list

documents = processed_twitter['Lemm_Tweets'].tolist()

# Tokenize and remove punctuation
translator = str.maketrans("", "", string.punctuation)
tokenized_documents = [
    [
        token.translate(translator)
        for token in nltk.word_tokenize(doc)
        if token.isalpha() and token not in stop_words
    ]
    for doc in documents
]

# Create a dictionary representation of the documents
dict = corpora.Dictionary(tokenized_documents)

# Filter out tokens that appear in less than 5 documents or more than 80% of the documents
dict.filter_extremes(no_below = 500, no_above = 0.8)

# Create a bag-of-words representation of the documents
corpus = [dict.doc2bow(doc) for doc in tokenized_documents]

# Train the LDA model
lda_model = models.LdaModel(corpus, num_topics = 10, id2word=dict, passes = 15)

# Print the topics and their top words
topics = lda_model.print_topics(num_words = 5)
for topic in topics:
    words = [word.split("*")[1].strip().strip('"') for word in topic[1].split("+")]
    print(' + '.join(words))

# Assign topics to each document
processed_twitter['LDA_Topic'] = [max(lda_model[doc], key=lambda x: x[1])[0] for doc in corpus]

book + open + wait + We + plane
get + like + Thanks + day + ask
flight + The + update + Please + security
airport + travel + security + Hi + morning
need + fly + Its + ask + book
I + love + travel + ask + book
hour + go + morning + park + back
please + time + take + You + today
people + terminal + one + queue + What
would + still + ask + like + book
